In [3]:
from sklearn.datasets import make_circles


ModuleNotFoundError: No module named 'sklearn'

In [ ]:
n_samples = 1000 
X, y = make_circles(n_samples, 
                   noise=0.03, 
                   random_state=42)

In [ ]:
import pandas as pd 

In [ ]:
circles = pd.DataFrame({"X1": X[:,0],
                    "X2": X[:,1],
                    "Label": y  } )
circles.head(10)

In [ ]:
circles.Label.value_counts()

In [ ]:
# Visualize with plot
import matplotlib.pyplot as plt 

plt.scatter(x=X[:,0], y=X[:,1],
            c=y,
           cmap=plt.cm.RdYlBu) # red , yellow blue
"""
# For binary classification (y = [0, 1]):
y = 0  →  Red color
y = 1  →  Blue color

# For 3-class classification (y = [0, 1, 2]):
y = 0  →  Red color
y = 1  →  Yellow color  
y = 2  →  Blue color
"""

## 1.1 Input and Output shape

In [ ]:
X.shape, y.shape

In [ ]:
# View the first example of features and labels
X_sample = X[0]
y_sample = y[0]
print(f"Values for one sample of X: {X_sample} and the same for y: {y_sample}")
print(f"Shapes for one sample of X: {X_sample.shape} and the same for y: {y_sample.shape}")

## 1.2 Turn data into tensors and create train and test splits

In [ ]:
import torch 
X = torch.from_numpy(X).type(torch.float)
y = torch.from_numpy(y).type(torch.float)

# View the first five samples
X[:5], y[:5]

In [ ]:
# Split data into train and test sets
from sklearn.model_selection import train_test_split 

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                   test_size=0.2,
                                                   random_state=42)
len(X_train), len(X_test), len(y_train), len(y_test)

## 2.Building a model

In [ ]:
import torch 
from torch import nn 

device = "cuda" if torch.cuda.is_available() else "cpu"
device

In [ ]:
# 1. Construct a model class that subclasses nn.Module
class CircleModelV0(nn.Module):
    def __init__(self):
        super().__init__()
        # 2. Create 2 nn.Linear layers capable of handling X and y input and output shapes
        self.layer1 = nn.Linear(in_features=2, out_features=5)
        self.layer2 = nn.Linear(in_features=5, out_features=1)
    def forward(self, x):
        return self.layer2(self.layer1(x))

model_0 = CircleModelV0().to(device)
model_0

In [ ]:
# You can also do the same as above using nn.Sequential.
model_0 = nn.Sequential(
    nn.Linear(in_features=2, out_features=5),
    nn.Linear(in_features=5, out_features=1)
).to(device)

model_0

## why not just always use nn.Sequential?
- nn.Sequential is fantastic for straight-forward computations, however, as the namespace says, it always runs in sequential order.
- So if you'd like something else to happen (rather than just straight-forward sequential computation) you'll want to define your own custom nn.Module subclass

In [ ]:
untrained_preds = model_0(X_test.to(device))
print(f"Length of predictions: {len(untrained_preds)}, Shape: {untrained_preds.shape}")
print(f"Length of test samples: {len(y_test)}, Shape: {y_test.shape}")
print(f"\nFirst 10 predictions:\n{untrained_preds[:10]}")
print(f"\nFirst 10 test labels:\n{y_test[:10]}")

### PyTorch has two binary cross entropy implementations:

- torch.nn.BCELoss() - Creates a loss function that measures the binary cross entropy between the target (label) and input (features).
- torch.nn.BCEWithLogitsLoss() - This is the same as above except it has a sigmoid layer (nn.Sigmoid) built-in (we'll see what this means soon).

In [ ]:
# Create a loss function 
loss_fn = nn.BCEWithLogitsLoss()

# Create an optimizer 
optimizer = torch.optim.SGD(params = model_0.parameters(),
                           lr=0.1)

In [ ]:
# Calculate accuracy (a classification metric)
def accuracy_fn(y_true, y_pred):
    correct = torch.eq(y_true, y_pred).sum().item() # torch.eq() calculates where two tensors are equal
    acc = (correct / len(y_pred)) * 100
    return acc


## 3. Train Model 

### 3.1 Going from raw model outputs to predicted labels (logits -> prediction probabilities -> prediction labels)

In [ ]:
y_logits = model_0(X_test.to(device))[:5] # the raw outputs of our model are often referred to as logits.
y_logits 

In [ ]:
# Use sigmoid on model logits
y_pred_probs = torch.sigmoid(y_logits)
y_pred_probs

In [ ]:
# Find the predicted labels (round the prediction probabilities)
y_preds = torch.round(y_pred_probs)

# Get rid of extra dimension
y_preds.squeeze()

In [ ]:
y_test[:5]

### 3.2 Building a training and testing loop

In [ ]:
torch.manual_seed(42)

# set the n of epochs 
epochs = 100 

# Put data to target device 
X_train, y_train = X_train.to(device), y_train.to(device)
X_test, y_test = X_test.to(device), y_test.to(device)

# Build training and evaluation loop
for epoch in range(epochs):
    ### Training 
    model_0.train()

    # 1. Forward pass (model outputs raw logits)
    y_logits = model_0(X_train).squeeze() # The raw, unnormalized outputs of the model, usually the direct output of the last linear (fully connected) layer.
    y_pred = torch.round(torch.sigmoid(y_logits)) # The final predictions of the model, which can mean: Probabilities (after applying softmax or sigmoid), etc

    # 2. Calculate loss/accuracy
    # loss = loss_fn(torch.sigmoid(y_logits), # Using nn.BCELoss you need torch.sigmoid()
    #                y_train) 
    loss = loss_fn(y_logits, # Using nn.BCEWithLogitsLoss works with raw logits
                   y_train) 
    acc = accuracy_fn(y_true=y_train, 
                      y_pred=y_pred) 

    # 3. loss backward
    loss.backward()

    # 4. Optimizer step 
    optimizer.step()

    # 5. Optimizer zero grad
    optimizer.zero_grad()

    ### Testing 
    model_0.eval()
    with torch.inference_mode():
        test_logits = model_0(X_test).squeeze()
        test_pred = torch.round(torch.sigmoid(test_logits))

        # 2. Caculate loss/accuracy
        test_loss = loss_fn(test_logits, y_test)
        test_acc = accuracy_fn(y_true=y_test, 
                              y_pred=test_pred)
        if epoch % 10 == 0:
            print(f"Epoch: {epoch} | Loss: {loss:.5f}, Accuracy: {acc:.2f}% | Test loss: {test_loss:.5f}, Test acc: {test_acc:.2f}%")